In [15]:
import os

In [16]:
pwd%

'c:\\Users\\PQC\\mlflow\\ml_project_with_mlflow\\research'

In [17]:
pwd

'c:\\Users\\PQC\\mlflow\\ml_project_with_mlflow\\research'

In [18]:
os.chdir("../")

In [31]:
from mlproject.constants import *
 

In [32]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [33]:
from mlproject.utils.common import read_yaml,create_directories
from mlproject.constants import *

In [34]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
            config = self.config.data_ingestion

            create_directories([config.root_dir])

            data_ingestion_config = DataIngestionConfig(
                root_dir=config.root_dir,
                source_URL=config.source_URL,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir 
            )

            return data_ingestion_config

In [35]:
import os
import urllib.request as request
import zipfile
from mlproject import logger
from mlproject.utils.common import get_size

In [36]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [37]:
try:
    config= ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

config1.yaml
[2023-10-27 11:17:07,396: INFO: common: yaml file: config1.yaml loaded successfully]
params.yaml
[2023-10-27 11:17:07,409: INFO: common: yaml file: params.yaml loaded successfully]
schema.yaml
[2023-10-27 11:17:07,413: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-27 11:17:07,414: INFO: common: created directory at: artifacts]
[2023-10-27 11:17:07,415: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-27 11:17:08,157: INFO: 2366278714: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6100:29C1:B462D2:DE170B:653B4EDB
Accep

In [38]:
from box import ConfigBox

def read_yaml(path_to_yaml: Path) -> ConfigBox:
    """reads yaml file and returns

    Args:
        path_to_yaml (str): path like input

    Raises:
        ValueError: if yaml file is empty
        e: empty file

    Returns:
        ConfigBox: ConfigBox type
    """
    try:
        with open(path_to_yaml) as yaml_file:
            print(path_to_yaml)
            content = yaml.safe_load(yaml_file)
            logger.info(f"yaml file: {path_to_yaml} loaded successfully")
            return ConfigBox(content)
    except BoxValueError:
        raise ValueError("yaml file is empty")
    except Exception as e:
        raise e

In [39]:
read_yaml(Path("config\config.yaml"))

config\config.yaml


NameError: name 'BoxValueError' is not defined

In [ ]:
pwd

'c:\\Users\\PQC\\mlflow\\ml_project_with_mlflow\\research'

In [ ]:
os.chdir("../")